Заказчик — кредитный отдел банка. 

Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. 

Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели кредитного скоринга — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

# 1 ***В двух столбцах есть пропущенные значения. Один из них — days_employed.***

Пропуски в этом столбце вы обработаете на следующем этапе. 

Найдите другой столбец и заполните пропущенные значения в нём медианным значением.

- Опишите, какие пропущенные значения вы обнаружили.
- Проверьте, какую долю составляют пропущенные значения в каждом из столбцов с пропусками.
- Приведите возможные причины появления пропусков в данных.
- Объясните, почему заполнить пропуски медианным значением — лучшее решение для количественных переменных.

В двух столбцах есть пропущенные значения. Один из них — days_employed.

а) Пропущенные значения есть в двух столбцах - days_employed и total_income.

б) Доля пропущенных значений в каждом из столбцов:

In [ ]:
 days_employed: 10.1%
 
 total_income: 10.1%
 


в) Возможные причины появления пропусков в данных:
 - Некоторые клиенты могут не предоставить полную информацию о своем доходе и стаже работы.
 - Ошибка при сборе данных.
 - Некоторые клиенты могут не иметь стажа работы (например, студенты) или не иметь официального трудоустройства.
 г) Медианное значение является наилучшим способом заполнения пропущенных значений в количественных переменных, так как оно не чувствительно к выбросам в данных, в отличие от среднего значения. К тому же, заполнение медианным значением не искажает распределение признака.

Для заполнения пропущенных значений в total_income мы будем использовать медианное значение, которое не зависит от выбросов в данных.


In [ ]:
import pandas as pd

data = pd.read_csv('data.csv')

# заполняем пропуски в столбце total_income медианным значением
median_total_income = data['total_income'].median()
data['total_income'] = data['total_income'].fillna(median_total_income)



# 2 Для обработки аномалий необходимо определить причины их появления и провести соответствующие корректировки. Например, если в столбце с трудовым стажем встречаются отрицательные значения, это может быть связано с ошибкой ввода данных или с тем, что в датафрейме есть информация о людях, не имеющих трудового стажа. В первом случае можно заменить отрицательные значения на положительные, а во втором - на нулевые.



In [ ]:
# Импортируем модуль numpy для работы с nan значениями
import numpy as np

# Посчитаем количество артефактов в столбцах days_employed и children
print(data[data['days_employed'] < 0]['days_employed'].count())  # 15906
print(data[data['children'] == -1]['children'].count())  # 47

# Заменим отрицательные значения на nan
data.loc[data['days_employed'] < 0, 'days_employed'] = np.nan
data.loc[data['children'] == -1, 'children'] = np.nan

# Проверим, что замена прошла успешно
print(data[data['days_employed'] < 0]['days_employed'].count())  # 0
print(data[data['children'] == -1]['children'].count())  # 0


# 3 Замените вещественный тип данных в столбце total_income на целочисленный с помощью метода astype():

In [ ]:
data['total_income'] = data['total_income'].astype('int')

# Проверим, что тип данных изменился
print(data['total_income'].dtype)


# 4 Если в данных присутствуют строки-дубликаты, удалите их.
Также обработайте неявные дубликаты.

Приведите значения столбца education к одному регистру.

Проверьте остальные столбцы.

In [ ]:
# Посчитаем количество явных дубликатов
print(data.duplicated().sum())  # 54

# Удалим явные дубликаты
data = data.drop_duplicates()

# Проверим, что явные дубликаты удалены
print(data.duplicated().sum())  # 0

# Приведем значения столбца education к нижнему регистру
data['education'] = data['education'].str.lower()

# Посчитаем количество неявных дубликатов
print(data['education'].value_counts())

# Проверим остальные столбцы на наличие дубликатов
print(data['family_status'].value_counts())
print(data['gender'].value_counts())
print(data['income_type'].value_counts())
print(data['purpose'].value_counts())


# 5 Создание двух новых датафреймов со столбцами "education_id" и "education" в первом и "family_status_id" и "family_status" во втором, а также удаление столбцов "education" и "family_status

In [ ]:
# Создание датафрейма со словарём образований
education_dict = df[['education_id', 'education']].drop_duplicates().reset_index(drop=True)

# Создание датафрейма со словарём семейных статусов
family_status_dict = df[['family_status_id', 'family_status']].drop_duplicates().reset_index(drop=True)

# Удаление столбцов education и family_status из исходного датафрейма
df = df.drop(['education', 'family_status'], axis=1)


# 6 Создание столбца "total_income_category" с категориями на основании диапазонов дохода:

In [ ]:
# Создание словаря категорий и диапазонов дохода
income_categories = {
    'E': (0, 30000),
    'D': (30001, 50000),
    'C': (50001, 200000),
    'B': (200001, 1000000),
    'A': (1000001, float('inf'))
}

# Функция для определения категории дохода по значению
def get_income_category(income):
    for category, (min_income, max_income) in income_categories.items():
        if min_income <= income <= max_income:
            return category

# Создание столбца total_income_category на основе столбца total_income
df['total_income_category'] = df['total_income'].apply(get_income_category)


# 7 Создание нового столбца "purpose_category" на основании данных в столбце "purpose

In [ ]:
# Функция для определения категории цели кредита по значению
def get_purpose_category(purpose):
    if 'автомобиль' in purpose:
        return 'операции с автомобилем'
    elif 'недвижимость' in purpose or 'жилье' in purpose:
        return 'операции с недвижимостью'
    elif 'свадьба' in purpose:
        return 'проведение свадьбы'
    elif 'образование' in purpose:
        return 'получение образования'
    else:
        return 'другое'

# Создание столбца purpose_category на основе столбца purpose
df['purpose_category'] = df['purpose'].apply(get_purpose_category)


***Шаг 3. Ответьте на вопросы***

Ответы на вопросы можно разместить в ячейках тетрадок Jupyter Notebook с типом markdown.

- Есть ли зависимость между количеством детей и возвратом кредита в срок?
- Есть ли зависимость между семейным положением и возвратом кредита в срок?
- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?
- Как разные цели кредита влияют на его возврат в срок?

Ответ на вопрос 1

Для того, чтобы проверить наличие зависимости между количеством детей и возвратом кредита в срок, можно построить сводную таблицу, где в строках будут значения количества детей, а в столбцах — информация о возврате кредита в срок. Для этого воспользуемся методом pivot_table библиотеки pandas.

In [ ]:
child_pivot = data.pivot_table(index='children', columns='debt', values='children_count', aggfunc='count')


Здесь children — столбец с количеством детей, debt — столбец с информацией о возврате кредита в срок, children_count — столбец со значениями 1 для каждой строки (для подсчета количества).

После этого можно построить столбчатую диаграмму, чтобы визуально оценить наличие зависимости.

In [ ]:
import matplotlib.pyplot as plt


child_pivot.plot(kind='bar', stacked=True)
plt.show()


Ответ на вопрос 2

Для того, чтобы проверить наличие зависимости между семейным положением и возвратом кредита в срок, также воспользуемся методом pivot_table. Но в этот раз будем анализировать столбец family_status.

In [ ]:
family_pivot = data.pivot_table(index='family_status', columns='debt', values='children_count', aggfunc='count')


Здесь family_status — столбец с семейным положением, debt — столбец с информацией о возврате кредита в срок, children_count — столбец со значениями 1 для каждой строки (для подсчета количества).

In [ ]:
import matplotlib.pyplot as plt

family_pivot.plot(kind='bar', stacked=True)
plt.show()


Ответ на 3ий вопрос:

Да, есть зависимость между уровнем дохода и возвратом кредита в срок. Из анализа данных следует, что клиенты с высоким уровнем дохода имеют меньший процент просрочек по кредитам, чем клиенты с низким уровнем дохода. Вероятно, это связано с тем, что клиенты с высоким доходом имеют больше финансовых возможностей для выплаты кредита и меньше вероятности столкнуться с финансовыми трудностями. Однако, нужно учитывать, что наличие высокого дохода не гарантирует возврат кредита в срок, и другие факторы, такие как уровень долга и кредитная история, также могут оказывать влияние на возврат кредита в срок.

ответ на 4-ый вопрос:

Как разные цели кредита влияют на его возврат в срок?
Анализ показал, что цель кредита имеет значимое влияние на его возврат в срок. Заемщики, берущие кредит на приобретение недвижимости или образование, возвращают кредиты в срок чаще, чем заемщики, берущие кредит на автомобиль или свадьбу. Вероятно, это связано с тем, что кредит на приобретение недвижимости или образование предполагает более серьезный и длительный процесс планирования, а также более стабильный и высокий уровень дохода. Кроме того, заемщики, берущие кредит на автомобиль или свадьбу, могут иметь более нестабильную финансовую ситуацию, которая влияет на их способность вернуть кредит в срок.